In [ ]:
import os
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.utils import class_weight
import math 

from keras.applications import xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers

!pip install split-folders #to slpit the dataset in training-validation-test
import splitfolders

In [ ]:
# Random fixed seed for reproducibility
seed = 2113

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Dataset and training folders
dataset_dir = '/kaggle/input/leafdataset'
train_dir = os.path.join(dataset_dir, 'training')
labels = [
    "Apple",
    "Blueberry",
    "Cherry",
    "Corn",
    "Grape",
    "Orange",
    "Peach",
    "Pepper",
    "Potato",
    "Raspberry",
    "Soybean",
    "Squash",
    "Strawberry",
    "Tomato"
]

In [ ]:
# Plot example images from dataset

num_row = len(labels)//2
num_col = len(labels)//num_row
fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
for i in range(len(labels)):
  if i < len(labels):
    class_imgs = next(os.walk('{}/{}/'.format(train_dir, labels[i])))[2]
    class_img = class_imgs[0]
    img = Image.open('{}/{}/{}'.format(train_dir, labels[i], class_img))
    ax = axes[i//num_col, i%num_col]
    ax.imshow(np.array(img))
    ax.set_title('{}'.format(labels[i]))
plt.tight_layout()
plt.show()

# Dataset preprocessing
We used the metohd 'ratio' of the splitfoldres library to divide the dataset into training, validation and test sets. We decided to keep a 70% of the dataset for the training in order to obtain a well-trained model, a validation of 20% in order to keep track of the model's behaviour on a different set of data during training and also, even if not required, we decided to use part of the data (10%) for testing the performance of model prediction.

In [ ]:
splitfolders.ratio("/kaggle/input/leafdataset/training", output="output", seed=seed, ratio=(.7, .2, .1), group_prefix=None) # default values

In [ ]:
# Dataset folders 
output_dir = './output'
training_dir = os.path.join(output_dir, 'train')
validation_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

In [ ]:
#Data Augmentation on training set (but not on validation and test set)
train_data_gen = ImageDataGenerator(rotation_range=0.3, 
                     height_shift_range=0.4,
                     width_shift_range=0.4,
                     zoom_range=0.4,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='reflect',
                     rescale=1/255)
valid_data_gen = ImageDataGenerator(rescale=1/255)
test_data_gen = ImageDataGenerator(rescale=1/255)

train_gen = train_data_gen.flow_from_directory(directory=training_dir, 
                                              target_size=(256, 256), 
                                              color_mode='rgb',
                                              classes=None, 
                                              batch_size=20, 
                                              shuffle=True, 
                                              seed=seed) 
valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir, 
                                              target_size=(256, 256),
                                              color_mode='rgb',
                                              classes=None,
                                              batch_size=20,
                                              shuffle=False,
                                              seed=seed)
test_gen = test_data_gen.flow_from_directory(directory=test_dir, 
                                              target_size=(256, 256),
                                              color_mode='rgb',
                                              classes=None,
                                              batch_size=20,
                                              shuffle=False,
                                              seed=seed)

In [ ]:
# Compute weigths for each class to handle the unbalanced dataset
class_weights = class_weight.compute_class_weight(
           'balanced',
            classes=np.unique(train_gen.classes), 
            y=train_gen.classes)

dataset_stats = {}

for i, label in enumerate(labels):
    files = os.listdir(os.path.join(train_dir, label))
    dataset_stats[i] = [label, len(files)]
df = pd.DataFrame.from_dict(dataset_stats, orient="index", columns=["Category", "Size"])
df["Weight"] = class_weights

df


In [ ]:
# Map class labels to weigths
weights={}

for i in range(len(class_weights)):
    weights[i] = class_weights[i]

weights


In [ ]:
input_shape = (256, 256, 3)
epochs = 400

# Transfer learning

In [ ]:
# Download and plot the Xception model
xception = tfk.applications.Xception(
    include_top=False, # we don't want to include the classifier 
    weights="imagenet", 
    input_shape=input_shape
)
xception.summary()
tfk.utils.plot_model(xception)


In [ ]:
# We freeze the convolutional layers in order to train only our classifier 
xception.trainable = False


# Classifier
inputs = tfk.Input(input_shape)
x = tfkl.Resizing(256, 256, interpolation="bicubic")(inputs)
x = xception(x)
#instead of flattening we use a global average pooling layer (better results, see reports for details)
x = tfkl.GlobalAveragePooling2D(name='GlobalPooling')(x) #global average pooling reduces the number of training parameters
x = tfkl.Dropout(0.3, seed=seed)(x) #dropout layer in order to reduce overfitting
x = tfkl.Dense(
    256, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
xe= tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
xe.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
xe.summary()


In [ ]:
# Train the model
xe_history = xe.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    class_weight = weights,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
xe.save("myModels/final0")

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(xe_history['loss'], label='Training', alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(xe_history['val_loss'], label='Validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(xe_history['accuracy'], label='Training', alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(xe_history['val_accuracy'], label='Validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()


# Fine Tuning

In [ ]:
# Set all Xception layers to True
xe.get_layer('xception').trainable = True
for i, layer in enumerate(xe.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)

In [ ]:
# Freeze first 65 layers to keep the most generic filters untouched
for i, layer in enumerate(xe.get_layer('xception').layers[:65]):
  layer.trainable=False
for i, layer in enumerate(xe.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)
xe.summary()


In [ ]:

# Compile the model
xe.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')


In [ ]:
# Utility function to create folders and callbacks (allows to log some informations at runtime during training) for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint: allows to automatically save the model during training
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts') 
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)


  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp'),#where i want to save the checkpoints
                                                     save_weights_only=False, 
                                                     save_best_only=False)  

  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                               profile_batch=0,
                                               histogram_freq=1) 
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=10, restore_best_weights=True)
  callbacks.append(es_callback)
    

  return callbacks

In [ ]:
mycallbacks = create_folders_and_callbacks(model_name='FinalModel')

In [ ]:
# Fine-tune the model
xef_history = xe.fit(
    x = train_gen,
    batch_size = 256,
    epochs = epochs,
    validation_data = valid_gen,
    class_weight = weights,
    callbacks = mycallbacks
).history

In [ ]:
# Plot the training of the fined-tuned model compared with the non fined-tuned one
plt.figure(figsize=(15,5))

plt.plot(xe_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(xe_history['val_loss'], label='Transfer Learning', alpha=.8, color='#ff7f0e')

plt.plot(xef_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(xef_history['val_loss'], label='Fine Tuning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))

plt.plot(xe_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(xe_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#ff7f0e')

plt.plot(xef_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(xef_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:

xe.save("myModels/f_final0")


# Testing the model

In [ ]:
# Predict the test set with the CNN
predictions = xe.predict(test_gen)
predictions.shape

In [ ]:
#Confution Matrix and Classification Report
Y_pred = xe.predict_generator(test_gen, 1786 // 20+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
plt.figure(figsize=(10,8))
cm = confusion_matrix(test_gen.classes, y_pred)
sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
print('Classification Report')
print(classification_report(test_gen.classes, y_pred, target_names=labels))

In [ ]:
# Plot the target images and the predictions
batch = next(test_gen)
batch_pr = xe.predict(batch[0])
ind=random.randint(0,19)
fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_size_inches(18,5)

image = batch[0][ind]
ax1.imshow(image)
cat_label = batch[1][ind]


label_index = np.argmax(cat_label)
ax1.set_title('True label: '+labels[label_index])
ax2.barh(labels, batch_pr[ind], color=plt.get_cmap('Paired').colors, log=True)
ax2.set_title('Predicted label: '+labels[np.argmax(batch_pr[ind])])
ax2.grid(alpha=.3)
plt.show()

# Visualization

In [ ]:
layers = [layer.output for layer in xe.layers[2].layers if isinstance(layer, tf.keras.layers.Conv2D)]
activation_model = tf.keras.Model(inputs=xe.layers[2].input, outputs=layers)
fmaps = activation_model.predict(tf.expand_dims(image, 0))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline
def display_activation(fmaps, depth=0, first_n=-1):

    fmaps = fmaps[depth] 
    if first_n > 0:
        fmaps = fmaps[0, :, :, :first_n] 
        fmaps = tf.image.resize(fmaps, size=[128, 128]) 

    # Distribute on a grid for plotting
    col_size = 8
    row_size = fmaps.shape[-1] // 8
    fmap_channel=0
    fig = plt.figure(figsize=(30, 30))
    grid = ImageGrid(fig, 111,  
                    nrows_ncols=(row_size, col_size),  
                    axes_pad=0.1,  
                    )
    for row in range(0,row_size):
        for col in range(0,col_size):
            grid[fmap_channel].imshow(fmaps[0, :, :, fmap_channel], cmap='gray', aspect='auto')
            fmap_channel += 1
    plt.show()

In [ ]:
display_activation(fmaps=fmaps, depth=1, first_n=-1)